In [2]:
import pandas as pd
import numpy as np
purprd = pd.read_csv('지역_시간/dataframe/purprd.csv')

In [3]:
pd.set_option('max_columns', None)

In [4]:
pr2 = purprd.query('period=="2014_2"')
pr3 = purprd.query('period=="2015_1"')

In [4]:
pr2

,channel,receipt_no,code_dae,code_jung,code_so,cust_no,branch_no,pur_date,pur_hour,pur_amount,cat_dae,cat_jung,cat_so,year,month,yoil,half,week,pur_time,period,pur_week_time
3,B,6956422,15,1504,B150401,13823,12,2014-08-13,18,2250,가공식품,탄산음료,사이다,2014,8,수,2,주중,오후,2014_2,주중_오후
4,B,6957156,15,1504,B150401,18970,12,2014-08-31,22,4900,가공식품,탄산음료,사이다,2014,8,일,2,주말,밤,2014_2,주말_밤
31,B,8353764,15,1504,B150401,10757,34,2014-08-05,20,1780,가공식품,탄산음료,사이다,2014,8,화,2,주중,밤,2014_2,주중_밤
32,B,9578488,15,1504,B150401,18775,76,2014-07-10,22,1250,가공식품,탄산음료,사이다,2014,7,목,2,주중,밤,2014_2,주중_밤
33,B,9893006,15,1504,B150401,15586,88,2014-10-27,14,1680,가공식품,탄산음료,사이다,2014,10,월,2,주중,오후,2014_2,주중_오후
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28593007,B,8223874,36,3605,B360504,14901,32,2014-10-09,23,65000,전문스포츠/레저,스포츠ACC,스포츠가방,2014,10,목,2,주중,밤,2014_2,주중_밤
28593009,B,8183981,45,4508,B450804,940,31,2014-09-17,17,25000,패션잡화,지갑,기타지갑,2014,9,수,2,주중,오후,2014_2,주중_오후
28593013,B,130596,35,3505,B350502,2657,5,2014-12-13,19,599000,전문스포츠/레저,윈터스포츠,스키/보드장비,2014,12,토,2,주말,밤,2014_2,주말_밤
28593025,B,7269975,14,1415,B141503,134,16,2014-11-30,15,8900,가공식품,전통과자,온라인팥빙수재료,2014,11,일,2,주말,오후,2014_2,주말_오후


In [17]:
def total_amount(df):
    origin = df.copy()
    df1 = origin.groupby('cust_no')['pur_amount'].sum().reset_index()
    return df1

def item_amount(df):
    origin = df.copy()
    df1 = origin.pivot_table(index='cust_no', columns='cat_dae', values='pur_amount', aggfunc='sum').reset_index()
    return df1

def amount_process(df, item_ = True):
    origin = df.copy()
    df1 = total_amount(origin)
    df1.columns = ['cust_no','total_amount']
    for i in ['A','B','C']:
        tmp_df = origin.query(f'channel=="{i}"')
        df2 = total_amount(tmp_df)
        df2.columns = ['cust_no',f'amount_{i}']
        df1 = pd.merge(df1, df2, on='cust_no', how='left')
        
        if item_:
            df2 = item_amount(tmp_df).fillna(0)
            df1 = pd.merge(df1, df2, on='cust_no', how='outer')
            df2.set_index('cust_no',inplace=True)
            df2 = df2.apply(lambda x : x/x.sum(), axis=1)
            df2.columns = list(map(lambda x : x+'_ratio' , df2.columns))
            df1 = pd.merge(df1, df2.reset_index(), on='cust_no', how='outer')
            
        for j in [1,2]:
            if j == 1 :
                tmp_df1 = tmp_df.loc[(tmp_df['month'] < 4) | ((6 < tmp_df['month'] ) & (tmp_df['month'] <10))]
            else :
                tmp_df1 = tmp_df.loc[((3 < tmp_df['month'] ) & (tmp_df['month'] < 7)) | (9 < tmp_df['month'] )]
            
            df2 = total_amount(tmp_df1)
            df2.columns = ['cust_no',f'amount_{i}_{j}']
            df1 = pd.merge(df1, df2, on='cust_no', how='outer')
            
    if item_:
        df2 = item_amount(origin).fillna(0)
        df1 = pd.merge(df1, df2, on='cust_no', how='outer')
        df2.set_index('cust_no',inplace=True)
        df2 = df2.apply(lambda x : x/x.sum(), axis=1)
        df2.columns = list(map(lambda x : x+'_ratio' , df2.columns))
        df1 = pd.merge(df1, df2.reset_index(), on='cust_no', how='outer')
        
    return df1

def amount_merge_df(df1, df2 ,on='cust_no' ,suffixes=['_1','_2']):
    tmp_df1 = amount_process(df1)
    tmp_df2 = amount_process(df2)
    total_df = pd.merge(tmp_df1, tmp_df2, on=on, suffixes = suffixes, how='outer')
    return total_df

In [8]:
def total_freq(df):
    origin = df.copy()
    df1 = origin.groupby('cust_no')['receipt_no'].nunique().reset_index()
    return df1

def freq_process(df, item_ = True):
    origin = df.copy()
    df1 = total_freq(origin)
    df1.columns = ['cust_no','total_freq']
    for i in ['A','B','C']:
        tmp_df = origin.query(f'channel=="{i}"')
        df2 = total_freq(tmp_df)
        df2.columns = ['cust_no',f'freq_{i}']
        df1 = pd.merge(df1, df2, on='cust_no', how='left')
        for j in [1,2]:
            if j == 1 :
                tmp_df1 = tmp_df.loc[(tmp_df['month'] < 4) | ((6 < tmp_df['month'] ) & (tmp_df['month'] <10))]
            else :
                tmp_df1 = tmp_df.loc[((3 < tmp_df['month'] ) & (tmp_df['month'] < 7)) | (9 < tmp_df['month'] )]
            
            df2 = total_freq(tmp_df1)
            df2.columns = ['cust_no',f'freq_{i}_{j}']
            df1 = pd.merge(df1, df2, on='cust_no', how='outer')
                    
    return df1

def freq_merge_df(df1, df2 ,on='cust_no' ,suffixes=['_1','_2']):
    tmp_df1 = freq_process(df1)
    tmp_df2 = freq_process(df2)
    total_df = pd.merge(tmp_df1, tmp_df2, on=on, suffixes = suffixes, how='outer')
    return total_df

In [14]:
def durables(series):
    data = np.where(series.isin( ['가구/인테리어', '가전', '명품']),'내구재',
            np.where(series.isin( ['가공식품', '교육/문화용품', '신선식품', '의약품']),'비내구재','내구재_외'))
    return data

def durable_good(df):
    origin = df.copy()
    df1 = origin.pivot_table(index='cust_no',columns='category3', values='pur_amount', aggfunc='sum').fillna(0)
    df1 = df1.apply(lambda x : x/x.sum(), axis=1)
    df1 = df1[['내구재','비내구재']].reset_index()
    df1.columns = ['cust_no', '내구재_ratio','비내구재_ratio']
    return df1

def ex_total_process(df):
    origin = df.copy()
    origin['category3'] = durables(origin['cat_dae'])
    df1 = durable_good(origin)
    df2 = origin.pivot_table(index='cust_no',columns='week', values='receipt_no', aggfunc='nunique').apply(lambda x : x/x.sum(), axis=1)
    df1 = pd.merge(df1, df2[['주말']].reset_index(), on='cust_no', how='outer')
    return df1

def ex_merge_df(df1, df2 ,on='cust_no' ,suffixes=['_1','_2']):
    tmp_df1 = ex_total_process(df1)
    tmp_df2 = ex_total_process(df2)
    total_df = pd.merge(tmp_df1, tmp_df2, on=on, suffixes = suffixes, how='outer')
    return total_df

In [10]:
def total_origin_df(df1, df2,df3=None, on='cust_no', suffixes=['_2','_3']):
    amount_df = amount_merge_df(df1, df2, on=on, suffixes=suffixes)
    freq_df = freq_merge_df(df1, df2, on=on, suffixes=suffixes)
    ex_df = ex_merge_df(df1, df2, on=on, suffixes = suffixes)
    
    total_df = pd.merge(amount_df, freq_df, on=on, how='outer')
    total_df = pd.merge(total_df, ex_df, on=on, how='outer')
                        
    return total_df.fillna(0)

-------------------------------------------------------origin_df--------------------------------------------------------------

In [ ]:
# PCA 코드
total_freq = pca_df_1[pca_df_1.columns[pca_df_1.columns.str.contains('total_freq')]]
freq_A = pca_df_1[pca_df_1.columns[pca_df_1.columns.str.contains('freq_A')]]
freq_B = pca_df_1[pca_df_1.columns[pca_df_1.columns.str.contains('freq_B')]]
freq_C = pca_df_1[pca_df_1.columns[pca_df_1.columns.str.contains('freq_C')]]

from sklearn.decomposition import PCA
pca_tot = PCA(n_components = 2)
pca_A = PCA(n_components = 2)
pca_B = PCA(n_components = 2)
pca_C = PCA(n_components = 2)

pca_tot.fit(total_freq)
pca_A.fit(freq_A)
pca_B.fit(freq_B)
pca_C.fit(freq_C)

PC_score_tot = pca_tot.transform(total_freq)
PC_score_A = pca_A.transform(freq_A)
PC_score_B = pca_B.transform(freq_B)
PC_score_C = pca_C.transform(freq_C)

# PCA의 eigen_vector
print('total_freq', pca_tot.explained_variance_ratio_.round(3))
print('A', pca_A.explained_variance_ratio_.round(3))
print('B', pca_B.explained_variance_ratio_.round(3))
print('C', pca_C.explained_variance_ratio_.round(3))

pca_tot_columns = ['pca_tot_columns1', 'pca_tot_columns2']
pca_tot_df = pd.DataFrame(PC_score_tot, columns = pca_tot_columns)
pca_A_columns = ['pca_A_columns1', 'pca_A_columns2']
pca_A_df = pd.DataFrame(PC_score_A, columns = pca_A_columns)
pca_B_columns = ['pca_B_columns1', 'pca_B_columns2']
pca_B_df = pd.DataFrame(PC_score_B, columns = pca_B_columns)
pca_C_columns = ['pca_C_columns1', 'pca_C_columns2']
pca_C_df = pd.DataFrame(PC_score_C, columns = pca_C_columns)

pca = pd.concat([pca_tot_df, pca_A_df, pca_B_df, pca_C_df], axis = 1)
pca

In [18]:
total_ = total_origin_df(pr2, pr3)

In [19]:
total_

,cust_no,total_amount_2,amount_A_2,가공식품_x_2,가구/인테리어_x_2,가전_x_2,교육/문화용품_x_2,기타_x_2,명품_x_2,신선식품_x_2,외식_x_2,의류_x_2,일상용품_x_2,전문스포츠/레저_x_2,패션잡화_x_2,가공식품_ratio_x_2,가구/인테리어_ratio_x_2,가전_ratio_x_2,교육/문화용품_ratio_x_2,기타_ratio_x_2,명품_ratio_x_2,신선식품_ratio_x_2,외식_ratio_x_2,의류_ratio_x_2,일상용품_ratio_x_2,전문스포츠/레저_ratio_x_2,패션잡화_ratio_x_2,amount_A_1_2,amount_A_2_2,amount_B_2,가공식품_y_2,가구/인테리어_y_2,가전_y_2,교육/문화용품_y_2,기타_y_2,명품_y_2,신선식품_y_2,외식_y_2,의류_y_2,의약품_x_2,일상용품_y_2,전문스포츠/레저_y_2,패션잡화_y_2,가공식품_ratio_y_2,가구/인테리어_ratio_y_2,가전_ratio_y_2,교육/문화용품_ratio_y_2,기타_ratio_y_2,명품_ratio_y_2,신선식품_ratio_y_2,외식_ratio_y_2,의류_ratio_y_2,의약품_ratio_x_2,일상용품_ratio_y_2,전문스포츠/레저_ratio_y_2,패션잡화_ratio_y_2,amount_B_1_2,amount_B_2_2,amount_C_2,가공식품_x_2,가구/인테리어_x_2,가전_x_2,교육/문화용품_x_2,신선식품_x_2,의류_x_2,의약품_y_2,일상용품_x_2,전문스포츠/레저_x_2,패션잡화_x_2,가공식품_ratio_x_2,가구/인테리어_ratio_x_2,가전_ratio_x_2,교육/문화용품_ratio_x_2,신선식품_ratio_x_2,의류_ratio_x_2,의약품_ratio_y_2,일상용품_ratio_x_2,전문스포츠/레저_ratio_x_2,패션잡화_ratio_x_2,amount_C_1_2,amount_C_2_2,가공식품_y_2,가구/인테리어_y_2,가전_y_2,교육/문화용품_y_2,기타_2,명품_2,신선식품_y_2,외식_2,의류_y_2,의약품_2,일상용품_y_2,전문스포츠/레저_y_2,패션잡화_y_2,가공식품_ratio_y_2,가구/인테리어_ratio_y_2,가전_ratio_y_2,교육/문화용품_ratio_y_2,기타_ratio_2,명품_ratio_2,신선식품_ratio_y_2,외식_ratio_2,의류_ratio_y_2,의약품_ratio_2,일상용품_ratio_y_2,전문스포츠/레저_ratio_y_2,패션잡화_ratio_y_2,total_amount_3,amount_A_3,가공식품_x_3,가구/인테리어_x_3,가전_x_3,교육/문화용품_x_3,기타_x_3,명품_x_3,신선식품_x_3,외식_x_3,의류_x_3,일상용품_x_3,전문스포츠/레저_x_3,패션잡화_x_3,가공식품_ratio_x_3,가구/인테리어_ratio_x_3,가전_ratio_x_3,교육/문화용품_ratio_x_3,기타_ratio_x_3,명품_ratio_x_3,신선식품_ratio_x_3,외식_ratio_x_3,의류_ratio_x_3,일상용품_ratio_x_3,전문스포츠/레저_ratio_x_3,패션잡화_ratio_x_3,amount_A_1_3,amount_A_2_3,amount_B_3,가공식품_y_3,가구/인테리어_y_3,가전_y_3,교육/문화용품_y_3,기타_y_3,명품_y_3,신선식품_y_3,외식_y_3,의류_y_3,의약품_x_3,일상용품_y_3,전문스포츠/레저_y_3,패션잡화_y_3,가공식품_ratio_y_3,가구/인테리어_ratio_y_3,가전_ratio_y_3,교육/문화용품_ratio_y_3,기타_ratio_y_3,명품_ratio_y_3,신선식품_ratio_y_3,외식_ratio_y_3,의류_ratio_y_3,의약품_ratio_x_3,일상용품_ratio_y_3,전문스포츠/레저_ratio_y_3,패션잡화_ratio_y_3,amount_B_1_3,amount_B_2_3,amount_C_3,가공식품_x_3,가구/인테리어_x_3,가전_x_3,교육/문화용품_x_3,신선식품_x_3,의류_x_3,의약품_y_3,일상용품_x_3,전문스포츠/레저_x_3,패션잡화_x_3,가공식품_ratio_x_3,가구/인테리어_ratio_x_3,가전_ratio_x_3,교육/문화용품_ratio_x_3,신선식품_ratio_x_3,의류_ratio_x_3,의약품_ratio_y_3,일상용품_ratio_x_3,전문스포츠/레저_ratio_x_3,패션잡화_ratio_x_3,amount_C_1_3,amount_C_2_3,가공식품_y_3,가구/인테리어_y_3,가전_y_3,교육/문화용품_y_3,기타_3,명품_3,신선식품_y_3,외식_3,의류_y_3,의약품_3,일상용품_y_3,전문스포츠/레저_y_3,패션잡화_y_3,가공식품_ratio_y_3,가구/인테리어_ratio_y_3,가전_ratio_y_3,교육/문화용품_ratio_y_3,기타_ratio_3,명품_ratio_3,신선식품_ratio_y_3,외식_ratio_3,의류_ratio_y_3,의약품_ratio_3,일상용품_ratio_y_3,전문스포츠/레저_ratio_y_3,패션잡화_ratio_y_3,total_freq_2,freq_A_2,freq_A_1_2,freq_A_2_2,freq_B_2,freq_B_1_2,freq_B_2_2,freq_C_2,freq_C_1_2,freq_C_2_2,total_freq_3,freq_A_3,freq_A_1_3,freq_A_2_3,freq_B_3,freq_B_1_3,freq_B_2_3,freq_C_3,freq_C_1_3,freq_C_2_3,내구재_ratio_2,비내구재_ratio_2,주말_2,내구재_ratio_3,비내구재_ratio_3,주말_3
0,1,34116312.0,33340300.0,914680.0,4770240.0,9062100.0,0.0,0.0,0.0,924860.0,702400.0,11842680.0,4534240.0,0.0,589100.0,0.027435,0.143077,0.271806,0.0,0.0,0.000000,0.027740,0.021068,0.355206,0.135999,0.000000,0.017669,10722160.0,22618140.0,376140.0,102660.0,0.0,0.0,0.0,0.0,0.0,49100.0,56400.0,0.0,8000.0,159980.0,0.0,0.0,0.272930,0.000000,0.0,0.0,0.0,0.0,0.130537,0.149944,0.000000,0.021269,0.425320,0.0,0.000000,4700.0,371440.0,399872.0,95150.0,0.0,0.0,0.0,291342.0,0.0,0.0,13380.0,0.0,0.0,0.237951,0.0,0.000000,0.0,0.728588,0.000000,0.0,0.033461,0.0,0.0,54690.0,345182.0,1112490.0,4770240.0,9062100.0,0.0,0.0,0.0,1265302.0,758800.0,11842680.0,8000.0,4707600.0,0.0,589100.0,0.032609,0.139823,0.265624,0.0,0.0,0.000000,0.037088,0.022242,0.347127,0.000234,0.137987,0.000000,0.017267,17898365.0,17382750.0,1019040.0,577250.0,291000.0,0.0,0.0,503500.0,744130.0,657620.0,10673900.0,1089240.0,599980.0,1227090.0,0.058624,0.033208,0.016741,0.0,0.0,0.028965,0.042809,0.037832,0.614051,0.062662,0.034516,0.070592,9007070.0,8375680.0,208136.0,41780.0,0.0,0.0,0.0,0.0,0.0,135356.0,31000.0,0.0,0.0,0.0,0.0,0.0,0.200734,0.000000,0.0,0.000000,0.0,0.0,0.650325,0.148941,0.000000,0.000000,0.000000,

In [49]:
pr1 = purprd.query('period=="2014_1"')
pr1.head(2)

,channel,receipt_no,code_dae,code_jung,code_so,cust_no,branch_no,pur_date,pur_hour,pur_amount,cat_dae,cat_jung,cat_so,year,month,yoil,half,week,pur_time,period,pur_week_time
0,B,8664000,15,1504,B150401,17218,44,2014-02-22,20,2420,가공식품,탄산음료,사이다,2014,2,토,1,주말,밤,2014_1,주말_밤
1,B,8664007,15,1504,B150401,11303,44,2014-02-22,13,2400,가공식품,탄산음료,사이다,2014,2,토,1,주말,오후,2014_1,주말_오후


In [ ]:
total_df = ex_merge_df(pr2, pr3)

In [43]:
total_df

,cust_no,내구재_ratio_1,비내구재_ratio_1,주말_1,주중_1,내구재_ratio_2,비내구재_ratio_2,주말_2,주중_2
0,1,0.405447,0.069931,0.255708,0.744292,0.076641,0.124870,0.345946,0.654054
1,2,0.052967,0.149268,0.287425,0.712575,0.126046,0.097901,0.349398,0.650602
2,3,0.025802,0.868765,0.190476,0.809524,0.000000,0.908793,0.138298,0.861702
3,4,0.099095,0.163018,0.161905,0.838095,0.000000,0.375650,0.186567,0.813433
4,5,0.095384,0.532985,0.253333,0.746667,0.000000,0.603551,0.358025,0.641975
...,...,...,...,...,...,...,...,...,...
19351,19283,NaN,NaN,NaN,NaN,0.000000,0.272737,0.403846,0.596154
19352,19350,NaN,NaN,NaN,NaN,0.000000,0.224232,0.162791,0.837209
19353,19381,NaN,NaN,NaN,NaN,0.027885,0.724929,0.245161,0.754839
19354,19382,NaN,NaN,NaN,NaN,0.002463,0.772179,0.232877,0.767123


In [51]:
total_df['비내구재_ratio_ch'] = total_df['비내구재_ratio_2'].fillna(0) - total_df['비내구재_ratio_1'].fillna(0)

In [52]:
total_df[['cust_no','비내구재_ratio_ch']].to_csv('dataframe/비내구재_12기.csv',index=False)

In [26]:
total_df.to_csv('dataframe/23기origin.csv',index=False)

In [30]:
pr2['pur_date'] = pd.to_datetime(pr2['pur_date'])

In [32]:
pr2['quarter'] = pr2['pur_date'].dt.quarter

In [33]:
pr2

,channel,receipt_no,code_dae,code_jung,code_so,cust_no,branch_no,pur_date,pur_hour,pur_amount,cat_dae,cat_jung,cat_so,year,month,yoil,half,week,pur_time,period,pur_week_time,quarter
3,B,6956422,15,1504,B150401,13823,12,2014-08-13,18,2250,가공식품,탄산음료,사이다,2014,8,수,2,주중,오후,2014_2,주중_오후,3
4,B,6957156,15,1504,B150401,18970,12,2014-08-31,22,4900,가공식품,탄산음료,사이다,2014,8,일,2,주말,밤,2014_2,주말_밤,3
31,B,8353764,15,1504,B150401,10757,34,2014-08-05,20,1780,가공식품,탄산음료,사이다,2014,8,화,2,주중,밤,2014_2,주중_밤,3
32,B,9578488,15,1504,B150401,18775,76,2014-07-10,22,1250,가공식품,탄산음료,사이다,2014,7,목,2,주중,밤,2014_2,주중_밤,3
33,B,9893006,15,1504,B150401,15586,88,2014-10-27,14,1680,가공식품,탄산음료,사이다,2014,10,월,2,주중,오후,2014_2,주중_오후,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28593007,B,8223874,36,3605,B360504,14901,32,2014-10-09,23,65000,전문스포츠/레저,스포츠ACC,스포츠가방,2014,10,목,2,주중,밤,2014_2,주중_밤,4
28593009,B,8183981,45,4508,B450804,940,31,2014-09-17,17,25000,패션잡화,지갑,기타지갑,2014,9,수,2,주중,오후,2014_2,주중_오후,3
28593013,B,130596,35,3505,B350502,2657,5,2014-12-13,19,599000,전문스포츠/레저,윈터스포츠,스키/보드장비,2014,12,토,2,주말,밤,2014_2,주말_밤,4
28593025,B,7269975,14,1415,B141503,134,16,2014-11-30,15,8900,가공식품,전통과자,온라인팥빙수재료,2014,11,일,2,주말,오후,2014_2,주말_오후,4
